In [2]:
!ls

sample_data


In [1]:
import os
ROOT = '/home/mav204/Documents/minor-project'
os.chdir(ROOT)
print(ROOT)

FileNotFoundError: [Errno 2] No such file or directory: '/home/mav204/Documents/minor-project'

In [ ]:
!ls -l augment/

total 124
-rw-r--r-- 1 mav204 mav204      0 Dec 10 15:24 __init__.py
drwxr-xr-x 2 mav204 mav204   4096 Dec 10 14:27 __pycache__
-rw-r--r-- 1 mav204 mav204 113829 Dec 10 15:31 augment.ipynb
-rw-r--r-- 1 mav204 mav204   1428 Dec 10 15:05 back_translation.py
-rw-r--r-- 1 mav204 mav204   3946 Dec 10 15:28 synonym_replacement.py


In [ ]:
from math import ceil
from misc.dataset_modifier import separate, save_json, get_json
from augment.back_translation import Backtranslation
from augment.synonym_replacement import SynonymAugmenter

misc.dataset_modifier 




/home/mav204/Documents/minor-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/mav204/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mav204/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
BT = Backtranslation()
SYN = SynonymAugmenter()

Device:  cpu


In [ ]:
train_data = get_json(f'./data/curated/train.json')
questions = get_json(f'./data/metadata/acceptable.json')

correct_baseline = 50
partially_correct_baseline = 50
incorrect_baseline = 25

lookup_table = {}

for q in questions:
    qc = questions[q]['count']  # shorthand

    c = qc['correct']
    p = qc['partially correct']
    i = qc['incorrect']

    lookup_table[q] = {
        'correct': ceil(max(0, correct_baseline - c) / c),
        'partially correct': ceil(max(0, partially_correct_baseline - p) / p),
        'incorrect': ceil(max(0, incorrect_baseline - i) / i)
    }


In [ ]:
qid = ''
no_of_augmentations = 0
sim_ub = .85
sim_lb = .7
augmentations = []

In [ ]:
Total_number_augmentation = 0
for i in list(lookup_table.values()):
    Total_number_augmentation+= (i['correct'] + i['partially correct'] + i['incorrect'])
print(Total_number_augmentation)

242


In [ ]:
temp = []
for record in train_data:
    qid = record['id'][-4:]
    num_augments = lookup_table[qid][record['verification_feedback']]

    for i in range(num_augments):
        answer = record['provided_answer']
        variant = BT.translate(answer)
        variant2 = SYN.augment(answer)

        sim1 = SYN.sim(answer, variant)
        sim2 = SYN.sim(answer, variant2)

        v1_score = SYN.deviation_score(sim=sim1)
        v2_score = SYN.deviation_score(sim=sim2)

        best_variant = variant if v1_score < v2_score else variant2
        temp.append(best_variant)
    
    for i in range(len(temp)):
        augmentations.append({
            'id': f'aug{i:02}'+record['id'],
            'question': record['question'],
            'reference_answer': record['reference_answer'],
            'provided_answer': temp[i],
            'answer_feedback': record['answer_feedback'],
            'verification_feedback': record['verification_feedback'],
            'max_score': record['max_score'],
            'normalized_score': record['normalized_score']
        })

Generating replacements...
Attempt: 1
Attempt: 2
Attempt: 3
Attempt: 4
Attempt: 5
Attempt: 6
Attempt: 7
Attempt: 8
Attempt: 9
Attempt: 10
Attempt: 11
Attempt: 12
Attempt: 13
Attempt: 14
Attempt: 15
Attempt: 16
Attempt: 17
Attempt: 18
Attempt: 19
Attempt: 20
Attempt: 21
Generating replacements...
Attempt: 1
0.8293404579162598
Generating replacements...
Attempt: 1
Attempt: 2
Attempt: 3
Attempt: 4
Attempt: 5
Attempt: 6
Attempt: 7
Attempt: 8
Attempt: 9
Attempt: 10
Attempt: 11
Attempt: 12
Attempt: 13
Attempt: 14
Attempt: 15
Attempt: 16
Attempt: 17
Attempt: 18
Attempt: 19
Attempt: 20
Attempt: 21
Generating replacements...
Attempt: 1
Attempt: 2
Attempt: 3
Attempt: 4
Attempt: 5
Attempt: 6
Attempt: 7
Attempt: 8
Attempt: 9
Attempt: 10
Attempt: 11
Attempt: 12
Attempt: 13
Attempt: 14
Attempt: 15
Attempt: 16
Attempt: 17
Attempt: 18
Attempt: 19
Attempt: 20
Attempt: 21
Generating replacements...
Attempt: 1
Attempt: 2
0.7837852835655212
Generating replacements...
Attempt: 1
Attempt: 2
Attempt: 3
Attem

KeyboardInterrupt: 

In [ ]:
save_json(augmentations, './data/augmented/train.json')